In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

all_data = pd.read_csv('game_data.csv')

In [37]:
all_data.home_win2

0       0.833333
1       0.900000
2       0.875000
3       0.916667
4       0.928571
5       0.928571
6       0.750000
7       0.800000
8       1.000000
9       0.928571
10      0.937500
11      0.933333
12      0.875000
13      0.937500
14      0.823529
15      0.823529
16      0.941176
17      0.916667
18      1.000000
19      0.937500
20      0.866667
21      0.923077
22      1.000000
23      0.916667
24      0.875000
25      0.846154
26      1.000000
27      0.933333
28      1.000000
29      1.000000
          ...   
1085    0.937500
1086    1.000000
1087    0.923077
1088    0.882353
1089    0.600000
1090    0.812500
1091    1.000000
1092    0.882353
1093    0.666667
1094    0.937500
1095    0.764706
1096    0.777778
1097    0.944444
1098    0.714286
1099    0.928571
1100    1.000000
1101    1.000000
1102    0.944444
1103    0.937500
1104    0.875000
1105    0.875000
1106    0.944444
1107    0.937500
1108    1.000000
1109    0.937500
1110    0.944444
1111    0.937500
1112    0.9375

In [2]:
corr_matrix = all_data.corr()
corr_matrix
corr_matrix['result'].sort_values(ascending=False)

result                      1.000000
seeds1                      0.418291
rank_year01                 0.351665
rank_year11                 0.333424
rank_year21                 0.324882
win2                        0.292535
away_win2                   0.226137
played_last_tour2           0.216560
highest_level_last_tour2    0.212036
loss1                       0.201788
wins_last_tour2             0.192173
home_win2                   0.191320
field_score2                0.186767
blocks2                     0.184204
score_av2                   0.153330
asists2                     0.145726
turn_overs1                 0.142685
offensive_rebound2          0.125422
defansive_rebound2          0.114414
score_ag_av1                0.112522
personal_fouls1             0.095443
steals2                     0.076313
previous_results            0.044632
3pointer_score2             0.017153
team2_id                   -0.004659
team1_id                   -0.007094
game_year                  -0.031569
g

In [7]:
import plotly.express as px
fig1 = px.scatter(all_data, x='win1' ,y='win2',color = 'result' )
fig2 = px.scatter(all_data, x='loss1' ,y='loss2',color = 'result' )
#fig1.show()

In [30]:
train_data = all_data[all_data['game_year']!=2019]
test_data = all_data[all_data['game_year'] == 2019]
train_feat = train_data.drop(['game_year','game_day','result','team1_id','team2_id'], axis = 1)
train_label = train_data.result
test_feat = test_data.drop(['game_year','game_day','result','team1_id','team2_id'], axis = 1)
test_label = test_data.result

In [4]:
train_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048 entries, 0 to 1047
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   win1                      1048 non-null   float64
 1   win2                      1048 non-null   float64
 2   loss1                     1048 non-null   float64
 3   loss2                     1048 non-null   float64
 4   home_win1                 1048 non-null   float64
 5   home_win2                 1048 non-null   float64
 6   away_win1                 1048 non-null   float64
 7   away_win2                 1048 non-null   float64
 8   rank_year01               1048 non-null   float64
 9   rank_year02               1048 non-null   float64
 10  score_av1                 1048 non-null   float64
 11  score_av2                 1048 non-null   float64
 12  score_ag_av1              1048 non-null   float64
 13  score_ag_av2              1048 non-null   float64
 14  wins_las

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

scaler = StandardScaler()
train_feat = scaler.fit_transform(train_feat)
test_feat = scaler.transform(test_feat)

In [19]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(train_feat,train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
accuracy_score(test_label,log_reg.predict(test_feat))

0.6716417910447762

In [21]:
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression()
param_grid_log = [{'C':np.geomspace(0.1,5,200),'solver':['lbfgs']},{'C':np.geomspace(0.1,5,200),'solver':['liblinear'],\
                                                                'penalty':['l1']}]
search_log = GridSearchCV(log_reg, param_grid_log, cv = 5, scoring = 'accuracy',verbose=0)
search_log.fit(train_feat, train_label)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': array([0.1       , 0.10...
       3.11938586, 3.18131473, 3.24447308, 3.3088853 , 3.37457628,
       3.44157143, 3.50989662, 3.57957827, 3.6506433 , 3.72311918,
       3.79703392, 3.87241607, 3.94929479, 4.02769976, 4.10766131,
       4.18921032, 4.27237832, 4.35719745, 4.44370048, 4.53192084,
       4.62189264

In [22]:
accuracy_score(test_label,search_log.predict(test_feat))

0.6716417910447762

In [8]:
scores = pd.DataFrame(search_log.cv_results_)
import plotly.express as px
fig_log = px.scatter(scores, x='param_C' ,y='mean_test_score',color = 'param_solver' )

In [8]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(dual = False)
linear_svc.fit(train_feat, train_label)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [9]:
accuracy_score(test_label,linear_svc.predict(test_feat))

0.746268656716418

In [10]:
from sklearn.svm import SVC
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
svm = SVC()
param_ls_svc = [{'kernel':['linear'], 'C':np.geomspace(0.1,5,10)}, \
                {'kernel':['poly'],'C':np.geomspace(0.1,5,10),'degree':range(1,10)},\
                {'kernel':['rbf'],'C':np.geomspace(0.1,5,10),'gamma':np.geomspace(0.1,5,10)}]
grid_svc = GridSearchCV(svm,param_ls_svc,cv = 5, scoring = 'accuracy',verbose= 10)
grid_svc.fit(train_feat, train_label)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.716, total=   0.1s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.719, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.646, total=   0.0s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.708, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.684, total=   0.0s
[CV] C=0.15444521049463789, kernel=linear ............................
[CV]  C=0.15444521049463789, kernel=linear, score=0.701, total=   0.0s
[CV] C=0.15444521049463789, kernel=linear ............................
[CV]  C=0.15444521049463789, kernel=linear, score=0.714, total=   0.1s
[CV] C=0.15444521049463789, kernel=linear ............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s


[CV]  C=0.15444521049463789, kernel=linear, score=0.646, total=   0.0s
[CV] C=0.15444521049463789, kernel=linear ............................
[CV]  C=0.15444521049463789, kernel=linear, score=0.708, total=   0.1s
[CV] C=0.15444521049463789, kernel=linear ............................
[CV]  C=0.15444521049463789, kernel=linear, score=0.684, total=   0.1s
[CV] C=0.23853323044733007, kernel=linear ............................
[CV]  C=0.23853323044733007, kernel=linear, score=0.711, total=   0.1s
[CV] C=0.23853323044733007, kernel=linear ............................


...

[CV]  C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf, score=0.521, total=   0.1s
[CV] C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf .......
[CV]  C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf, score=0.519, total=   0.1s
[CV] C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf .......
[CV]  C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf .......
[CV]  C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf .......
[CV]  C=2.096144000826768, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=2.096144000826768, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=2.096144000826768, gamma=5.000000000000001, kernel=rbf, score=0.521, total=   0.1s
[CV] C=2.096144000826768, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=2.0961440008267

[CV]  C=3.2373940143476263, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=3.2373940143476263, gamma=3.2373940143476263, kernel=rbf ......
[CV]  C=3.2373940143476263, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=3.2373940143476263, gamma=3.2373940143476263, kernel=rbf ......
[CV]  C=3.2373940143476263, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf .......
[CV]  C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf, score=0.521, total=   0.1s
[CV] C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf .......
[CV]  C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf, score=0.519, total=   0.1s
[CV] C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf .......
[CV]  C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf, score=0.522, total=   0.1s
[CV] C=3.2373940143476263, gamma=5.000000000000001, kernel=rbf .......
[CV]  C=3.237394014

[CV]  C=5.000000000000001, gamma=3.2373940143476263, kernel=rbf, score=0.522, total=   0.1s
[CV] C=5.000000000000001, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=5.000000000000001, gamma=5.000000000000001, kernel=rbf, score=0.521, total=   0.1s
[CV] C=5.000000000000001, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=5.000000000000001, gamma=5.000000000000001, kernel=rbf, score=0.519, total=   0.1s
[CV] C=5.000000000000001, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=5.000000000000001, gamma=5.000000000000001, kernel=rbf, score=0.522, total=   0.1s
[CV] C=5.000000000000001, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=5.000000000000001, gamma=5.000000000000001, kernel=rbf, score=0.522, total=   0.1s
[CV] C=5.000000000000001, gamma=5.000000000000001, kernel=rbf ........
[CV]  C=5.000000000000001, gamma=5.000000000000001, kernel=rbf, score=0.522, total=   0.1s


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': array([0.1       , 0.15444521, 0.23853323,...
                          'degree': range(1, 10), 'kernel': ['poly']},
                         {'C': array([0.1       , 0.15444521, 0.23853323, 0.36840315, 0.56898102,
       0.87876393, 1.35720881, 2.096144  , 3.23739401, 5.        ]),
                          'gamma': array([0.1       , 0.15444521, 0.23853323, 0.36840315, 0.56898102,
       0.87876393, 1.35720881, 2.096144  , 3.23739401, 5.        ]),
                          'kernel': ['rbf']}],
             pr

In [11]:
accuracy_score(test_label,grid_svc.predict(test_feat))

0.6865671641791045

In [12]:
from sklearn.ensemble import RandomForestClassifier

ran_for = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
ran_for.fit(train_feat,train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [13]:
accuracy_score(test_label,ran_for.predict(test_feat))

0.6716417910447762

In [18]:
feat_score_list = list()
i = 0
for score in ran_for.feature_importances_:
    feat_score_list.append([list(train_feat.columns)[i],score])
    i+=1

In [19]:
from operator import itemgetter
feature_ranking = sorted(feat_score_list, key=itemgetter(1))
feature_ranking

[['3pointer_score__against_av2', 0.005436714882984225],
 ['ast_av2', 0.00662762649383968],
 ['off_rb_av1', 0.007142360480490897],
 ['field_score_against1', 0.0073222178546723275],
 ['highest_level_last_tour2', 0.0073736772105303695],
 ['home_win2', 0.0073932544578078924],
 ['3pointer_score_av2', 0.007728198891113255],
 ['3pointer_score__against_av1', 0.007966632960752456],
 ['def_rba_av2', 0.00814910524916679],
 ['wins_last_tour2', 0.0081993330548308],
 ['def_rba_av1', 0.008739040755211423],
 ['score_av2', 0.009049529706731828],
 ['foul_av2', 0.009166826701087826],
 ['away_win2', 0.009207144585245502],
 ['3pointer_score_av1', 0.009474908280593122],
 ['ast_av1', 0.009547266984289007],
 ['wins_last_tour1', 0.00972749054540362],
 ['field_score_against2', 0.010006491304447983],
 ['field_score_av2', 0.010056418654378141],
 ['highest_level_last_tour1', 0.010077920009478943],
 ['steal_av2', 0.010411729174092969],
 ['block_av2', 0.011014815893208687],
 ['home_win1', 0.011100107864227328],
 ['o

In [14]:
param_grid_rf = {'bootstrap':[True],
                'max_depth':[80,90,100,110],
                'max_features':[2,3],
                'min_samples_leaf':[3,4,5],
                'min_samples_split':[8,10,12],
                'n_estimators':[100,200,300,1000]}
ran_for_sc = RandomForestClassifier()
search_ran_for = GridSearchCV(ran_for_sc,param_grid_rf,cv=5, scoring='accuracy',verbose=2)
search_ran_for.fit(train_feat,train_label)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samp

...

[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=200, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=4, min_samples_split=12, n_estimators=300 
[CV]  bootstrap=True, max_depth=110, max_fea

[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_fea

[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.3s
[CV] bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000 
[CV]  bootstrap=True, max_depth=110, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000, total=   2.2s


[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed: 22.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [15]:
accuracy_score(test_label,search_ran_for.predict(test_feat))

0.7313432835820896

In [23]:
search_log.best_params_

{'C': 0.3317338586271527, 'penalty': 'l1', 'solver': 'liblinear'}

In [36]:
ran_for = RandomForestClassifier(bootstrap =True ,
                max_depth =110,
                min_samples_leaf = 4,
                min_samples_split = 10,
                n_estimators = 200)
ran_for.fit(train_feat,train_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=110, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
accuracy_score(test_label,ran_for.predict(test_feat))

0.7164179104477612

In [24]:
search_log.best_params_

{'C': 0.3317338586271527, 'penalty': 'l1', 'solver': 'liblinear'}

In [55]:
log_reg = LogisticRegression(C=0.3317338586, solver='liblinear',penalty = 'l1')
log_reg.fit(train_feat,train_label)

LogisticRegression(C=0.27, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
accuracy_score(test_label,log_reg.predict(test_feat))

0.8059701492537313

In [31]:
from sklearn.ensemble import BaggingClassifier

bag_log_reg = BaggingClassifier(LogisticRegression(C=0.3317338586271527, solver='liblinear',penalty = 'l1'),
                                n_estimators=500,max_samples=300,bootstrap= True,n_jobs=-1)
bag_log_reg.fit(train_feat,train_label)

BaggingClassifier(base_estimator=LogisticRegression(C=0.3317338586271527,
                                                    class_weight=None,
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='warn',
                                                    n_jobs=None, penalty='l1',
                                                    random_state=None,
                                                    solver='liblinear',
                                                    tol=0.0001, verbose=0,
                                                    warm_start=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=300, n_estimators

In [32]:
accuracy_score(test_label,bag_log_reg.predict(test_feat))

0.7910447761194029

In [27]:
from sklearn.ensemble import VotingClassifier

log_reg = LogisticRegression(C=0.27, solver='liblinear',penalty = 'l1')
liner_svm = LinearSVC(dual=False)
ran_for = RandomForestClassifier(bootstrap=True,max_depth=110,min_samples_leaf=4,min_samples_split=10,
                                    n_estimators=200)

voting_cs = VotingClassifier(estimators = [('log_vt', log_reg),\
                                           ('ran_for_vt',ran_for),\
                                           ('linear_svm_vt', liner_svm)])
voting_cs.fit(train_feat,train_label)

VotingClassifier(estimators=[('log_vt',
                              LogisticRegression(C=0.27, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('ran_for_vt',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None...
                                                     n_estimators=200,
                       

In [28]:
accuracy_score(test_label,voting_cs.predict(test_feat))

0.6716417910447762

In [63]:
accuracy_score(test_label,bag_log_reg.predict(test_feat_sc))

0.7164179104477612